## Preprocessing

## Optimization 4th attempt

### In this optimization attempt, I will be increasing the number of values for each bin and changing the # of neurons in the hidden layers.

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv(r"C:\Users\tsatr\Deep_Learning_Challenge\Resources\charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN'], 1)
application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
application_df = application_df.drop(['NAME'], 1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

#### APPLICATION_TYPE,  CLASSIFICATION, and ASK_AMOUNT have more than 10 unique values.  

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

#set cutoff to 100, create dictionary from value_counts result then initialize list for app types to replace
cutoff = 100
app_type_count_dict = dict(application_type_counts)
application_types_to_replace = []

#iterate through items in dictionary and add app types with value <cutoff to list
for key, value in app_type_count_dict.items():
    if value < cutoff:
        application_types_to_replace.append(key)

#create copy of df for reduced application types
red_application_type_df = application_df

# Replace in dataframe
for app in application_types_to_replace:
    red_application_type_df['APPLICATION_TYPE'] = red_application_type_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
red_application_type_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = red_application_type_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1900        1
C1732        1
C4200        1
C1370        1
C5200        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts.loc[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1257        5
C1600        5
C2710        3
C1260        3
C0           3
C1234        2
C3200        2
C1256        2
C1246        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#set cutoff to 200, create dictionary from value_counts result then initialize list for classifications to replace

cutoff = 200
classification_counts_dict = dict(classification_counts)
classifications_to_replace = []

#iterate through items in dictionary, add classifications with value <cutoff to list
for key, value in classification_counts_dict.items():
    if value < cutoff:
        classifications_to_replace.append(key)
        
#create copy of dataframe for reduced classifications
red_classifications_df = red_application_type_df

# Replace in dataframe
for cls in classifications_to_replace:
    red_classifications_df['CLASSIFICATION'] = red_classifications_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
red_classifications_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(red_classifications_df)
dummies_df.head

<bound method NDFrame.head of        STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                       0   
1           1    108590              1                       0   
2           1      5000              0                       0   
3           1      6692              1                       0   
4           1    142590              1                       0   
...       ...       ...            ...                     ...   
34294       1      5000              0                       0   
34295       1      5000              0                       0   
34296       1      5000              0                       0   
34297       1      5000              1                       0   
34298       1  36500179              0                       0   

       APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                         1                     0                    0   
1                         0  

In [11]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop('IS_SUCCESSFUL', axis = 1)
y = dummies_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test, = train_test_split(X, y, random_state=42)

In [12]:
print('X_train:\t{}'.format(X_train.shape))
print('y_train:\t{}'.format(y_train.shape))
print('X_test:\t{}'.format(X_test.shape))
print('y_test:\t{}'.format(y_test.shape))

X_train:	(25724, 46)
y_train:	(25724,)
X_test:	(8575, 46)
y_test:	(8575,)


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
hidden_nodes_layer1=500
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=46))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               23500     
                                                                 
 dense_1 (Dense)             (None, 50)                25050     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 48,601
Trainable params: 48,601
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
import os
#Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/", exist_ok=True)
checkpoint_dir = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

from keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint

# Create a callback that saves the model's weights every 5 epochs
checkpoint = ModelCheckpoint(filepath = checkpoint_dir, monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=5)


In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 831us/step - loss: 0.5676 - accuracy: 0.7238
Epoch 2/50
804/804 [==============================] - 1s 816us/step - loss: 0.5541 - accuracy: 0.7271
Epoch 3/50
804/804 [==============================] - 1s 799us/step - loss: 0.5515 - accuracy: 0.7298
Epoch 4/50
804/804 [==============================] - 1s 815us/step - loss: 0.5495 - accuracy: 0.7300
Epoch 5/50
804/804 [==============================] - 1s 803us/step - loss: 0.5483 - accuracy: 0.7311
Epoch 6/50
804/804 [==============================] - 1s 797us/step - loss: 0.5477 - accuracy: 0.7317
Epoch 7/50
804/804 [==============================] - 1s 803us/step - loss: 0.5471 - accuracy: 0.7327
Epoch 8/50
804/804 [==============================] - 1s 807us/step - loss: 0.5466 - accuracy: 0.7328
Epoch 9/50
804/804 [==============================] - 1s 803us/step - loss: 0.5449 - accuracy: 0.7340
Epoch 10/50
804/804 [==============================] - 1s 807us/step - loss: 0.545

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5595 - accuracy: 0.7321 - 222ms/epoch - 829us/step
Loss: 0.5594511032104492, Accuracy: 0.7321282625198364


In [19]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization4.h5')

### The fourth optimization achieved an accuracy rate of 73.0%.  
